# Ethereum ELT

In [1]:
import os
import json
import requests
import datetime

from web3 import Web3
import pandas as pd
import numpy as np

#### Alchemy as the remote node provider

In [2]:
ALCHEMY_KEY = os.environ.get('KEY')
w3 = Web3(Web3.HTTPProvider('https://eth-mainnet.alchemyapi.io/v2/'+ALCHEMY_KEY))

In [3]:
url = 'https://eth-mainnet.alchemyapi.io/v2/'+ALCHEMY_KEY

In [86]:
latest_block_int = w3.eth.blockNumber
previous_block_int = latest_block_int - 1

In [87]:
block_num_request_data = {"jsonrpc": "2.0","id": 0,"method": "eth_getBlockByNumber","params":[hex(previous_block_int),True]}
block_json = requests.post(url, json=block_num_request_data).json()
block_list = block_json['result']['transactions']

first_rows = len(block_list)
print(f'{previous_block_int}: {first_rows}')

block_int = previous_block_int
while block_int > previous_block_int - 50:
    block_int -= 1
    block_num_request_data = {"jsonrpc": "2.0","id": 0,"method": "eth_getBlockByNumber","params":[hex(block_int),True]}
    block_json = requests.post(url, json=block_num_request_data).json()
    rows = len(block_json['result']['transactions'])
    print(f'{block_int}: {rows}')
    block_list.extend(block_json['result']['transactions'])
    
total_rows = len(block_list)
print(f'total rows: {total_rows}')

15055507: 34
15055506: 217
15055505: 131
15055504: 233
15055503: 366
15055502: 326
15055501: 61
15055500: 52
15055499: 322
15055498: 301
15055497: 354
15055496: 308
15055495: 77
15055494: 257
15055493: 313
15055492: 84
15055491: 23
15055490: 159
15055489: 83
15055488: 123
15055487: 151
15055486: 94
15055485: 99
15055484: 283
15055483: 110
15055482: 39
15055481: 193
15055480: 83
15055479: 24
15055478: 221
15055477: 24
15055476: 380
15055475: 65
15055474: 249
15055473: 437
15055472: 175
15055471: 138
15055470: 312
15055469: 40
15055468: 234
15055467: 132
15055466: 262
15055465: 263
15055464: 0
15055463: 92
15055462: 99
15055461: 21
15055460: 170
15055459: 116
15055458: 80
15055457: 458
total rows: 8868


In [124]:
block_num_request_data = {"jsonrpc": "2.0","id": 0,"method": "eth_getBlockByNumber","params":[hex(previous_block_int),False]}
block_json = requests.post(url, json=block_num_request_data).json()

In [125]:
date_time = dt.datetime.fromtimestamp(int(block_json['result']['timestamp'],16))
prev_block_dt_str = date_time.strftime('%Y-%m-%d %H:%M:%S')
prev_block_dt_str

'2022-07-01 08:18:15'

In [126]:
block_num_request_data = {"jsonrpc": "2.0","id": 0,"method": "eth_getBlockByNumber","params":[hex(block_int),False]}
block_json = requests.post(url, json=block_num_request_data).json()

In [127]:
date_time = dt.datetime.fromtimestamp(int(block_json['result']['timestamp'],16))
earliest_block_dt_str = date_time.strftime('%Y-%m-%d %H:%M:%S')
earliest_block_dt_str

'2022-07-01 08:07:09'

#### Export and save as files ...